In [1]:
%config IPCompleter.greedy=True

In [2]:
import os
import pandas as pd

data_dir = "../data/"
datasets_fn = [
    (data_dir + file)
    for file in os.listdir(data_dir)
]

datasets = [
    pd.read_csv(file_name)
    for file_name in datasets_fn
]

datasets[0]

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
0,3,2001/12/12 17:41,1,1,3,0.579380,0.093620,0.000000,480.666667,1,14,596,0.666667,143,133.500000,129,1
1,7,1999/10/12 12:57,1,1,1,0.000000,0.000000,0.000000,398.000000,1,1,0,1.000000,140,140.000000,137,1
2,8,2002/5/15 16:55,3,3,52,0.739279,0.183477,0.208913,283.519231,0,23,15836,0.750000,984,818.650000,978,0
3,9,2002/1/21 15:37,1,1,8,0.685328,0.016039,0.012880,514.375000,1,21,1281,1.000000,579,479.250000,550,0
4,10,2001/12/19 16:44,2,2,38,0.769776,0.091829,0.072746,366.815789,1,21,6565,0.763158,413,313.250000,405,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [3]:
from sklearn.model_selection import KFold

kf_splitter = KFold(n_splits=10, shuffle=False, random_state=None)

def split_train_data(dataset):
    """Returns all the experiments with each experiment represented
    as a tuple: (training data, testing data)"""
    return [
        (dataset.iloc[train_indicies], dataset.iloc[test_indicies])
        for train_indicies, test_indicies in kf_splitter.split(dataset)
    ]

# Contains (train, test) for all data sets
splitted_data = [
    split_train_data(data)
    for data in datasets
]

data_train, data_test = splitted_data[0][0]
data_train

,transactionid,commitdate,ns,nm,nf,entropy,la,ld,lt,fix,ndev,pd,npt,exp,rexp,sexp,bug
462,1012,2002/7/6 5:33,1,1,1,0.000000,0.021978,0.000000,182.000000,1,20,3,1.000000,7,7.000000,7,1
463,1014,2002/6/14 12:56,1,1,1,0.000000,0.005525,0.005525,181.000000,1,19,37,1.000000,88,68.000000,86,0
464,1015,2002/5/8 17:25,1,1,1,0.000000,0.005525,0.000000,181.000000,0,19,0,1.000000,180,156.333333,173,0
465,1016,2002/5/8 15:02,1,1,1,0.000000,0.005556,0.011111,180.000000,1,19,14,1.000000,179,155.333333,172,0
466,1017,2002/4/24 3:24,2,2,28,0.934459,0.021663,0.029374,194.535714,1,22,974,0.785714,118,94.333333,118,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,20762,2002/7/26 16:44,1,1,1,0.000000,0.025698,0.039106,895.000000,1,21,0,1.000000,219,190.083333,212,1
4616,20840,2002/11/13 3:29,1,1,2,0.918296,0.005450,0.002725,183.500000,1,18,21,1.000000,355,252.833333,335,1
4617,20936,2005/9/27 12:54,1,1,1,0.000000,0.000000,0.000000,2.000000,1,2,0,1.000000,961,239.861905,3,0
4618,20937,2005/9/27 13:03,1,1,1,0.000000,21.000000,0.000000,2.000000,1,2,0,1.000000,615,464.500000,3,0


In [7]:
def split_input_output(data):
    """Split the data into the information and the answer"""
    X = data[['lt', 'la']]
    Y = data['bug']
    return (X, Y)

input_output_data = [
    [
        (split_input_output(train_data), split_input_output(test_data))
        for train_data, test_data in dataset
    ]
    for dataset in splitted_data
]

# First dataset, First experiment, Training data, input
input_output_data[0][0][0][0]

,lt,la
462,182.000000,0.021978
463,181.000000,0.005525
464,181.000000,0.005525
465,180.000000,0.005556
466,194.535714,0.021663
...,...,...
4615,895.000000,0.025698
4616,183.500000,0.005450
4617,2.000000,0.000000
4618,2.000000,21.000000


In [25]:
# decChurn <- function(data){
# nf  <- data$nf             
# lt  <- data$lt * nf
# lt[lt==0] <- lt[lt==0] + 1
# churn <- ((data$la + data$ld) * lt)/2  # LA and LD was normalized by LT

# return (churn)
# }

### evaluate the prediction performance when considering effort (effort=churn)
# fit$bugdensity <- fit$bug/(churn.fit+1)
# est$bugdensity <- est$bug/(churn.est+1)


# density
# data$NUM/(data$LOC+1)

# Rank entries as follows:
# 1. LT (number of lines above)
# 2. Defect dencity
# 3. Larger change size
def sort_prediction_data(x, y):
    df = x
    df = df.assign(y=y)
    df = df.sort_values(by=['lt'])
    
#     print(df)
    
    y_out = df['y']
    x_out = df.drop('y', axis=1)
    
    return (x_out, y_out)

sorted_data = [
    [
        [
            sort_prediction_data(x, y)
            for x, y in experiment # training and testing
        ]
        for experiment in dataset
    ]
    for dataset in input_output_data
]

# a, b = sort_prediction_data(sorted_data[0][0][0][0], sorted_data[0][0][0][1])
# a

sorted_data[0][0][0][0]

,lt,la
4619,0.0,0.000000
3256,0.0,0.000000
2910,0.0,0.000000
4505,0.0,0.000000
2926,0.0,0.000000
...,...,...
878,2706.0,0.060606
874,2717.0,0.000736
873,2717.0,0.000368
983,2744.0,0.207726


In [26]:
# The next step is to predict using this knowledge in some way.
# We might be able to just use the number of expected bugs.

expected_bugs = {
    '../data/bugzilla.csv': 0.36,
    '../data/columba.csv': 0.14,
    '../data/jdt.csv': 0.14,
    '../data/mozilla.csv': 0.25,
    '../data/platform.csv': 0.05,
    '../data/postgres.csv': 0.2,
}

datasets_fn
for dataset in sorted_data:

['../data/bugzilla.csv',
 '../data/columba.csv',
 '../data/jdt.csv',
 '../data/mozilla.csv',
 '../data/platform.csv',
 '../data/postgres.csv']